# Build an N-gram Language Model

## Definition

A language model is a probability distribution over sequences of a vocabulary.

Let **V** be a vocabulary, items could be characters, tokens or words. 

A language model assigns a probability to each sequence of items:

- A language model LM should predict the next word of a sequence of words (based on the known history).
- We would like a language model to assign higher probabilities to sentences that are real and syntactically correct.

Let $(w_1, w_2, ... ,w_n)$ be a sequence of words from vocabulary **V** and **LM** a language model over **V** with probablity **P** then:

 $P(w_{n+1}|w_1, ... ,w_n) = LM(w_{n+1})$.


In [1]:
#IPython Magic: https://ipython.readthedocs.io/en/stable/interactive/magics.html
%load_ext autoreload
%autoreload 2

## Data: Load some data

Load the book Alice in Wonderland from Gutenberg: https://www.gutenberg.org/ebooks/11

In [2]:
from urllib.request import urlretrieve

url="https://www.gutenberg.org/files/11/11-0.txt"
filename = "./data/ALICE/alice.txt"

urlretrieve(url=url, filename=filename) #Retrive a URL into a file on disk


with open(filename, "r") as f:
    data = f.read()
    print(len(data))
    start = data.find("[Illustration]")
    end = data.find("*** END OF THE PROJECT GUTENBERG EBOOK")

    data = data[start+len("[Illustration]"):end]
    print(len(data))




144694
144586


## Corps: Build a simple corpus with NLTK

A corpus is a list of sentences. Each sentence s a list of words.

In [3]:
!pip install nltk

In [4]:
import nltk 

# nltk.sent_tokenize needs resource punkt_tab. Use the NLTK Downloader to obtain the resource:
nltk.download('punkt_tab')

corpus = [] #corpus as a list of list

sentences = nltk.sent_tokenize(data) 

print(f"{len(sentences)} sentences found")

for sentence in sentences:
    tokens = nltk.word_tokenize(sentence)
    corpus.append(tokens)



985 sentences found


[nltk_data] Downloading package punkt_tab to /Users/done/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Vocabulary: Build a vocabulary

A vocabulary is a set of words

In [5]:
vocabulary = set()
words = []
for s in corpus:
    for w in s:
        vocabulary.add(w)
        words.append(w)
print(f"The book Alice in wonderland contains {len(corpus)} sentences with {len(words)} words.")   
print(f"A language model based on the book has a vocabulray with {len(vocabulary)} different words.")

The book Alice in wonderland contains 985 sentences with 34636 words.
A language model based on the book has a vocabulray with 3284 different words.


## Uni-gram
Uni-gram is a LM where each word is independent of all other words. There is no history:

$$ P(w_1, w_2, ... ,w_n) = P(w_1) * P(w_2)* ... * P(w_n)$$

In [6]:
from collections import Counter
counter = Counter(words) # Counter counts the occurence of each item in a list of items.

uni_gram = {w: counter[w]/len(words) for w in counter}

print(f"The probability of the word Alice is P('Alice') = {uni_gram['Alice']}")
print(f"The sentence 'Alice lives in Wonderland' has the Probability P('Alice')*P('lives')*P('in')*P('Wonderland') \
= {uni_gram['Alice']*uni_gram['lives']*uni_gram['in']*uni_gram['Wonderland']}")


print(f"OCC('Alice')      = {counter['Alice']}")
print(f"OCC('lives')      = {counter['lives']}")
print(f"OCC('in')         = {counter['in']}")
print(f"OCC('Wonderland') = {counter['Wonderland']}")


print(f"P('Alice')        = {uni_gram['Alice']}")
print(f"P('lives')        = {uni_gram['lives']}")
print(f"P('in')           = {uni_gram['in']}")
print(f"P('Wonderland')   = {uni_gram['Wonderland']}")


The probability of the word Alice is P('Alice') = 0.01149093428802402
The sentence 'Alice lives in Wonderland' has the Probability P('Alice')*P('lives')*P('in')*P('Wonderland') = 1.1747803029063786e-12
OCC('Alice')      = 398
OCC('lives')      = 4
OCC('in')         = 354
OCC('Wonderland') = 3
P('Alice')        = 0.01149093428802402
P('lives')        = 0.00011548677676406051
P('in')           = 0.010220579743619356
P('Wonderland')   = 8.661508257304539e-05


## N-gram
N-gram is a Language Model that knows all sequences up to length N:
$$ P(w_1, w_2, ... ,w_n) = P(w_1) * P(w_2|w_1)* ... * P(w_n|w_1, ... ,w_{n-1})$$
We can compute the probabilty for the next word given a history $(w_1, w_2, ... ,w_{n-1})$.

In [7]:
# Use '#' as separator
"#" in vocabulary

False

## Build a 2-gram LM based on the corpus of alice

Use 2-item sequences separated by '#'  

In [8]:
# get 2-item sequences
bi_gram = {}
x = 0
for i in range(len(words)-1):
    seq = "#".join(words[i:i+2])
    x += 1
    if seq not in bi_gram:
        bi_gram[seq] = 0
    bi_gram[seq] += 1
bi_gram = {w: bi_gram[w]/x for w in bi_gram}    

### Build a 3-gram LM based on the corpus of alice

Use 3-item sequences separated by '#'  

In [9]:
tri_gram = {}
x = 0
for i in range(len(words)-2):
    seq = "#".join(words[i:i+3])
    x += 1
    if seq not in tri_gram:
        tri_gram[seq] = 0
    tri_gram[seq] += 1
tri_gram = {w: tri_gram[w]/x for w in tri_gram}

In [10]:
# Get all 3-grams starting with "Alice was" to compute P("Alice", "was")
x = 0
seqs = []
seq = "#".join(['Alice','was'])

print(f"P('Alice', 'was') is {bi_gram[seq]} using the 2-gram LM")


for s in tri_gram:
    if s.startswith(seq):
        print(s, tri_gram[s])
        seqs.append(s.split("#"))
        x += tri_gram[s]
print(f"P('Alice', 'was') is {x} using the 3-gram LM")

P('Alice', 'was') is 0.000490832972426736 using the 2-gram LM
Alice#was#beginning 5.774672287347693e-05
Alice#was#not 8.662008431021539e-05
Alice#was#soon 2.8873361436738465e-05
Alice#was#so 2.8873361436738465e-05
Alice#was#more 2.8873361436738465e-05
Alice#was#just 2.8873361436738465e-05
Alice#was#a 2.8873361436738465e-05
Alice#was#silent 2.8873361436738465e-05
Alice#was#rather 2.8873361436738465e-05
Alice#was#very 5.774672287347693e-05
Alice#was#too 2.8873361436738465e-05
Alice#was#thoroughly 2.8873361436738465e-05
Alice#was#only 2.8873361436738465e-05
P('Alice', 'was') is 0.0004908471444245538 using the 3-gram LM


In [11]:
uni_gram["Alice"]

0.01149093428802402

In [12]:
test = 0 # the sum of all conditional probabilities should be 1
for _seq in seqs:
    p123 = tri_gram["#".join(_seq)]
    p1 = uni_gram[_seq[0]]
    p12 = bi_gram["#".join(_seq[0:2])]
    p21 = p12/p1
    p312 = p123/(p1*p21)
    test += p312
    print(f"P('{_seq[2]}'|'{_seq[0]}','{_seq[1]}')",p312) #, f"={p123}/({p1}*{p21})")
print(test)

P('beginning'|'Alice','was') 0.11765045568958078
P('not'|'Alice','was') 0.17647568353437118
P('soon'|'Alice','was') 0.05882522784479039
P('so'|'Alice','was') 0.05882522784479039
P('more'|'Alice','was') 0.05882522784479039
P('just'|'Alice','was') 0.05882522784479039
P('a'|'Alice','was') 0.05882522784479039
P('silent'|'Alice','was') 0.05882522784479039
P('rather'|'Alice','was') 0.05882522784479039
P('very'|'Alice','was') 0.11765045568958078
P('too'|'Alice','was') 0.05882522784479039
P('thoroughly'|'Alice','was') 0.05882522784479039
P('only'|'Alice','was') 0.05882522784479039
1.0000288733614366


## Exercise: Make a next word predictor based on bi-gram and tri-gram
#
# 1. Build a next word predictor based on bi-gram model
# 2. Build a next word predictor based in tri-gram model
# Think about what happend if a probailiy is zero?
# 3. Build a generation method that writes a little story based on bi- and tri-gram

In [14]:
############ bi-gram ###############
#
# p(x_1, x_2) = p(x_1) * p(x_2|x_1)
# notation: p12 = p1 * p21
#
####################################

def predict_next_bigram_word(start):
    seqs = []
    seq = start
    for s in bi_gram:
        if s.startswith(f"{seq}#"):
            p12 = bi_gram[s]
            p1 = uni_gram[start]
            p21 = p12/p1

            seqs.append((s.split("#"), p21))
            
    seqs = sorted(seqs, key=lambda x:x[1], reverse=True)
    if len(seqs) > 0:
        s, p = seqs[0]   
        return s[1], p
    else:
        return None, 1

In [15]:
predict_next_bigram_word("was")

('a', 0.08000230980222318)

In [16]:
def predict_next_word(history):
    seqs = []
    seq = "#".join(history)
    for s in tri_gram:
        if s.startswith(f"{seq}#"):
            
            p123 = tri_gram[s]
            p1 = uni_gram[history[0]]
            p12 = bi_gram["#".join(history)]
            p21 = p12/p1
            p312 = p123/(p1*p21)

            seqs.append((s.split("#"), p312))
            
    seqs = sorted(seqs, key=lambda x:x[1], reverse=True)
    if len(seqs) > 0:
        s, p = seqs[0]   
        return s[2], p
    else:
        return None, 1

In [18]:
predict_next_word(['Alice', 'was'])

('not', 0.17647568353437118)

In [19]:
from random import shuffle
def make_some_story(story:list, uni_gram, max_len=10):
    if len(story) == 0:
        init = list(uni_gram.keys())
        shuffle(init)
        story.append(init[0])
    if len(story) == 1:
        next, _ = predict_next_bigram_word(story[0])
        story.append(next)
    history = [story[-2], story[-1]]
    #print(history)
    next, _ = predict_next_word(history)
    story.append(next)
    if len(story) < max_len:
       make_some_story(story=story, uni_gram=uni_gram, max_len=max_len)
    

In [21]:
story = []
make_some_story(story=story, uni_gram=uni_gram, max_len=10)
print(story)

['usually', 'bleeds', ';', 'and', 'the', 'Queen', ',', 'who', 'was', 'trembling']
